<a href="https://colab.research.google.com/github/seismosmsr/machine_learning/blob/main/summarize-chatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# for google colab you have to install this 2 library before run the code
!pip install pypdf2
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
import os
import PyPDF2
import re
import openai
import pandas as pd
import json
import csv
import json


In [4]:
import zipfile
import os

def zip_specific_csv_files_in_directory(directory_path, output_zipfile, starting_string):
    with zipfile.ZipFile(output_zipfile, 'w') as zipf:
        for filename in os.listdir(directory_path):
            if filename.startswith(starting_string) and filename.endswith('.csv'):
                filepath = os.path.join(directory_path, filename)
                if os.path.isfile(filepath):  # Ensure it's a file and not a directory
                    zipf.write(filepath, os.path.basename(filepath))  # Only write the base filename

    print(f"CSV files in '{directory_path}' starting with '{starting_string}' zipped to '{output_zipfile}'")

# Example usage


In [5]:
import zipfile
import os

def zip_selected_files(directory_path, output_zipfile, file_list):
    with zipfile.ZipFile(output_zipfile, 'w') as zipf:
        for filename in file_list:
            filepath = os.path.join(directory_path, filename)
            if os.path.isfile(filepath):  # Ensure it's a file and not a directory
                zipf.write(filepath, os.path.basename(filepath))

    print(f"Selected files from '{directory_path}' zipped to '{output_zipfile}'")

# Example usage
# file_list = ['file1.txt', 'file2.csv', 'image.jpg']


In [7]:
# Download pdf direcly
!curl -o paper.pdf https://anr.vermont.gov/sites/anr/files/specialtopics/climate/documents/VTCCwhitepapers/VTCCAdaptAgriculture.pdf #change the url with pdf which you wanna download

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  416k  100  416k    0     0   814k      0 --:--:-- --:--:-- --:--:--  815k


In [8]:
# Set the string that will contain the summary
pdf_summary_text = ""

# Open the PDF file
pdf_file_path = "paper.pdf"

# Read the PDF file using PyPDF2
pdf_file = open(pdf_file_path, 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)

In [12]:
openai.api_key = "na" #change the api key with yours

In [102]:
# for file_path in files:
#     try:
#         os.remove(file_path)
#         print(f"Successfully deleted {file_path}")
#     except OSError as e:
#         print(f"Error deleting {file_path}: {e}")

In [95]:
pdf_summary_text = ""
# Loop through all the pages in the PDF file
# range(len(pdf_reader.pages))
for page_num in range(len(pdf_reader.pages)):
    # Extract the text from the page
    page_text = pdf_reader.pages[page_num].extract_text().lower()


    paragraphs = [line.strip() for line in page_text.split('\n \n') if line.strip()]
    paragraphs_text = [f"Paragraph {i+1}:\n{paragraph}\n" for i, paragraph in enumerate(paragraphs)]

    for para_num, para_text in enumerate(paragraphs_text):
    # while bad_read:
      response = openai.ChatCompletion.create(
                      model="gpt-3.5-turbo",
                      messages=[
                          {"role": "system", "content": "You are a helpful research assistant. Specfically, you are helping research the impact of climate change on global food systems. We are doing a literature review. We want to know what crops and food systems are being impacted by specific climate change hazards. Only respond in the form of comma seperated values (csv). You always return a correctly formatted csv."},
                          {"role": "user", "content": f"Summarize the following text as a JSON document. Do not reply with anything except a JSON document. Please identify any specific climate change hazards, what cropping or food systems they effect, where globally the impact will be experienced, the specific quote from the paragraph (no more than 100 characters), if the impact is generally positive or negative (sentiment), and approximately what magnitude (e.g. high medium low). Do not do anything that could possibly break JSON formatting. Please make sure that text entries do not use commas internal to any text entries in the table. Please only return a JSON. The elements should only be: region, cropping_system, impact, sentiment, magnitude, quote, page_number (you are working on page {page_num}), paragraph_number (you are working on paragraph {para_num}. Here is the text:{para_text}. Do not return anything but the properly formatted JSON. It is of the utmost importance that the response you give is a properly formatted JSON. If the paragraph does not contain any text about climate change hazards, return an empty JSON (two empty curly braces) "},
                          ])

      page_summary = response["choices"][0]["message"]["content"]

      pdf_summary_text+=page_summary + "\n"

      pdf_summary_file = pdf_file_path.replace(os.path.splitext(pdf_file_path)[1], f"_summary.txt")

      with open(pdf_summary_file, "w+") as file:
          file.write(pdf_summary_text)

      pdf_summary_file = pdf_file_path.replace(os.path.splitext(pdf_file_path)[1], f"_{page_num}_{para_num}_summary.json")

      with open(pdf_summary_file, "w+") as file:
          file.write(page_summary)

# Close the PDF file
pdf_file.close()

with open(pdf_summary_file, "r") as file:
    print(file.read())

{}


In [103]:
import os
import json
import pandas as pd

# List of files
files = [f for f in os.listdir('/content/') if f.endswith('.json')]

# List to store dataframes
dfs = []

# List to store filenames of failed JSONs
failed_files = []

# Read each file and convert to dataframe
for file in files:
    try:
        with open(os.path.join('/content/', file), 'r') as f:
            data = json.load(f)

            # If the JSON data is a list of dictionaries
            if isinstance(data, list) and all(isinstance(item, dict) for item in data):
                df = pd.DataFrame(data)
                dfs.append(df)

            # If the JSON data is a single dictionary
            elif isinstance(data, dict):
                df = pd.DataFrame([data])
                dfs.append(df)

            # Handle other structures as needed
            else:
                print(f"Unhandled data structure in {file}. Skipping.")

    except json.JSONDecodeError:
        print(f"Failed to decode JSON for {file}. Adding to failed list.")
        failed_files.append(file)

# Combine all dataframes
combined_df = pd.concat(dfs, ignore_index=True)

# Write combined dataframe to CSV
combined_df.to_csv('merged_output.csv', index=False)

# If there were any failed files, save them to a CSV
if failed_files:
    failed_df = pd.DataFrame(failed_files, columns=["Failed Filenames"])
    failed_df.to_csv('failed_files.csv', index=False)

In [99]:
# files

In [100]:
zip_selected_files('/content/', 'jsons.zip', files)


Selected files from '/content/' zipped to 'jsons.zip'


In [101]:
zip_selected_files('/content/', 'results_3.zip', ['paper.pdf','failed_files.csv','merged_output.csv','jsons.zip'])


Selected files from '/content/' zipped to 'results_3.zip'
